In [298]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from bs4 import BeautifulSoup
from utils import *

path_2004 = "./2004/"
all_2004 = os.listdir(path_2004)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [301]:
results = []
for i, fname in enumerate(all_2004):
    print("File: ", i)
    test_f = open(path_2004 + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1]  

    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    multiplier = 1 # 1 by default
    
    table = scrape_table(wholetext)
    if table:
        tbl_flag = 1
        rows = scrape_rows(table)
        multiplier = get_multiplier_from_tbl(table)
    else:
        lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
            multiplier = get_multiplier_from_tbl_list(tbl_list)

            
    for row in rows:
        row_flag = 1
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                   '<1': row['<1'], '1-3':row['1-3'], '3-5':row['3-5'], '>5':row['>5'], 
                  'category': row['category'], 'multiplier': multiplier}
        if 'total' in row.keys():
                result['total'] = row['total']
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-', 'multiplier': multiplier}

        results.append(result)
pd.DataFrame(results).to_csv('scraped_files_df')

File:  0


/Users/divyas/Documents/G1/sec/utils.py:152: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 152 of the file /Users/divyas/Documents/G1/sec/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(wholetext)


Found a table
File:  1
Found a table
File:  2
Found a table
File:  3
Found a table
File:  4
Found a table
File:  5
Found a table
File:  6
Found a table
File:  7
Found a table
File:  8
Found a table
File:  9
Found a table
File:  10
Found a table
File:  11
Found a table
File:  12
Found a table
File:  13
Found a table
File:  14
Found a table
File:  15
Found a table
File:  16
Found a table
File:  17
Found a table
File:  18
Found a table
File:  19
Found a table
File:  20
Found a table
File:  21
Found a table
File:  22
Found a table
File:  23
Found a table
File:  24
Found a table
File:  25
Found a table
File:  26
Found a table
File:  27
Found a table
File:  28
Found a table
File:  29
Found a table
File:  30
Found a table
File:  31
Found a table
File:  32
Found a table
File:  33
Found a table
File:  34
Found a table
File:  35
Found a table
File:  36
Found a table
File:  37
Found a table
File:  38
Found a table
File:  39
Found a table
Unknown error:  could not convert string to float: '(0.3'
U

In [302]:
results_df = pd.DataFrame(results)
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print(len(set(last_check['fname'])), len(set(row_check['fname'])), len(set(none_check['fname'])))

49 7 0


In [303]:
last_check

,1-3,3-5,<1,>5,category,cik,datadate,fname,multiplier,row,tbl,total
1,27,-,12,0 0,capital lease\n obligations,0000881464,20031231,0001104659-04-007271.txt,1000,1,1,39
2,9220,10935,3073,31413,operating\n leases,0000881464,20031231,0001104659-04-007271.txt,1000,1,1,54641
3,321,463,513,1969,operating leases,0000018072,20040930,0001047469-04-035525.txt,1,1,1,672
4,270890,95387,113500,619155,gas supply,0000018072,20040930,0001047469-04-035525.txt,1,1,1,139378
5,55728,294147,28571,407017,transportation,0000018072,20040930,0001047469-04-035525.txt,1,1,1,28571
6,1723,1431,963,9480,operating leases(2),0001173284,20031231,0001193125-04-051895.txt,1,1,1,13597
7,20141,12090,12380,14710,noncancellable operating leases,0000790051,20031231,0001104659-04-006998.txt,1000,1,1,59321
8,2183,911,1224,1808,operating leases \n(1),0000007332,20031231,0000007332-04-000020.txt,1000,1,1,6126
9,4.774e+06,3.868e+06,2.752e+06,21321000,operating lease obligations,0001070772,20040630,0001104659-04-029370.txt,1,1,1,32715000
10,27,14,16,260,capital lease obligations,0001159297,20031231,0001193125-04-030895.txt,1000000,1,1,317


# Examining one file

In [295]:
test_f = open(path_2004 + "0001104659-04-007074.txt")
wholetext = test_f.read()
wholetext = wholetext.lower()
table = scrape_table(wholetext)

Found a table


[['(in\xa0millions) total',
  'less\xa0than\n  1\xa0year',
  '1\xa0-\xa03\xa0years',
  '3\xa0-\xa05\xa0years',
  'after\n  5\xa0years'],
 ['notes payable\n  obligations $', '2.5', '$', '2.5', '$ \x97 $ \x97 $ \x97'],
 ['non-cancelable\n  capital lease obligations',
  '0.1',
  '0.1',
  '\x97 \x97 \x97'],
 ['non-cancelable\n  operating lease obligations\x97equipment',
  '0.1',
  '\x97',
  '0.1',
  '\x97 \x97'],
 ['non-cancelable\n  operating lease obligations\x97buildings',
  '28.7',
  '3.6',
  '8.5',
  '8.8',
  '7.8'],
 ['long-term vendor\n  accounts payable', '0.7', '0.5', '\x97 \x97', '0.2'],
 ['open purchase\n  order commitments', '30.4', '23.2', '7.2', '\x97 \x97'],
 ['total contractual cash\n  obligations $',
  '62.5',
  '$',
  '29.9',
  '$',
  '15.8',
  '$',
  '8.8',
  '$',
  '8.0']]